In [100]:
import pyodbc
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import win32com.client as win32

In [101]:
today = date.today().strftime('%Y-%m-%d')
today

'2024-03-19'

In [102]:
# testing
# today = '2024-03-18'

In [103]:
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_tbl_ops_shortage_ext where ReportDate = '{today}'")
shortage_ext = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])

In [104]:
shortage_ext.loc[100,'P1'] = 100
shortage_ext.loc[100,'Working on upside'] = 500
shortage_ext.loc[100,'BT shortage'] = 500

check last FD date is empty

In [105]:
lastFDnull = shortage_ext[shortage_ext['last FD date'].isnull()].copy()
lastFDnull.loc[:,'Error'] = 'Last FD is blank'

print('Last FD is blank:', len(lastFDnull))

Last FD is blank: 1


P1~P3 sum <> total shortage

In [106]:
QtyMismatch = []
for i, row in shortage_ext.iterrows():
    P123 = row['P1'] + row['Net P2'] + row['Net P3']
    total = row['Total Shortage Qty'] 
    if P123 != total:
        QtyMismatch.append(row)
    else:
        pass

if len(QtyMismatch) > 0:
    df_QtyMismatch = pd.DataFrame(QtyMismatch)
    df_QtyMismatch['Error'] = 'P1~P3 sum <> total shortage'
else:
    df_QtyMismatch = pd.DataFrame()

print(f'P1~P3 sum != total shortage: {len(QtyMismatch)}')


P1~P3 sum != total shortage: 1


BT > P1

In [107]:
BT_P1 = []
for i, row in shortage_ext.iterrows():
    BT = row['BT shortage']
    P1 = row['P1'] 
    if BT > P1:
        BT_P1.append(row)
    else:
        pass

if len(BT_P1) > 0:
    df_BT_P1 = pd.DataFrame(BT_P1)
    df_BT_P1['Error'] = 'BT shortage > P1'
else:
    df_BT_P1 = pd.DataFrame()

print('BT shortage > P1:', len(BT_P1))


BT shortage > P1: 1


BT > total shortage

In [108]:
BT_total = []
for i, row in shortage_ext.iterrows():
    BT = row['BT shortage']
    total = row['Total Shortage Qty'] 
    if BT > total:
        BT_total.append(row)
    else:
        pass

if len(BT_total) > 0:
    df_BT_total = pd.DataFrame(BT_total)
    df_BT_total['Error'] = 'BT shortage > Total shortage'
else:
    df_BT_total = pd.DataFrame()

print('BT > total:', len(BT_total))

BT > total: 1


working on upside > total shortage

In [109]:
upside_total = []
for i, row in shortage_ext.iterrows():
    upside = row['Working on upside']
    total = row['Total Shortage Qty'] 
    if upside > total:
        upside_total.append(row)
    else:
        pass

if len(upside_total) > 0:
    df_upside_total = pd.DataFrame(upside_total)
    df_upside_total['Error'] = 'Working on upside > Total shortage'
else:
    df_upside_total = pd.DataFrame()

print('upside > total:', len(upside_total))

upside > total: 1


In [110]:
try:
    error_all = pd.concat([lastFDnull, df_QtyMismatch, df_BT_P1, df_BT_total, df_upside_total], axis=0)
    error_all['P1'] = error_all['P1'].astype(int)
    error_all['Net P2'] = error_all['Net P2'].astype(int)
    error_all['Net P3'] = error_all['Net P3'].astype(int)
    error_all['BT shortage'] = error_all['BT shortage'].astype(int)
    error_all['Working on upside'] = error_all['Working on upside'].astype(int)
    error_all['Total Shortage Qty'] = error_all['Total Shortage Qty'].astype(int)
except:
    error_all = pd.DataFrame()
    print('No error found')


In [111]:
if len(error_all) > 0:
    To_list = 'gps.taiwan.nb.buy-sell@hp.com'
    CC_list = 'spencer.cheng1@hp.com; dustin.hsu1@hp.com'
    mail = win32.Dispatch("Outlook.Application").CreateItem(0)
    mail.To = To_list
    mail.CC = CC_list
    mail.Subject = '<' + today + '> Shortage dashboard issue'
    message = 'Hi team,<br><br>\
    Please find below issues found in the latest shortage data.<br> Please use transformation tool to revise your data.<br><br>' + error_all.to_html(index=False) 
    mail.GetInspector 
    index = mail.HTMLbody.find('>', mail.HTMLbody.find('<body')) 
    mail.HTMLBody = mail.HTMLbody[:index + 1] + message + mail.HTMLbody[index + 1:] 
    mail.display()
else:
    print('No error found, no email sent')